In [2]:
pip install -qU langchain_community tiktoken langchainhub chromadb langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import getpass

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = getpass.getpass()

 ········


In [4]:
pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

 ········


In [7]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [9]:
pip install -qU langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [22]:
os.environ['OPENAI_API_KEY'] = getpass.getpass()

 ········


In [23]:
from langchain_openai import OpenAIEmbeddings


In [11]:
#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [13]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [24]:
# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [25]:
# Prompt
prompt = hub.pull("rlm/rag-prompt")

In [29]:
prompt2 = '''You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:'''
prompt2

"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

In [30]:
# LLM
# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a complex task into smaller and simpler steps. This is done by instructing the model to "think step by step" and decomposing the task into manageable sub-tasks, allowing for more effective planning and execution.'

In [31]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [32]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [33]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

1536

In [35]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8806581695120702


In [42]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)


# Make splits
splits = text_splitter.split_documents(docs)

In [48]:
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_kwargs={"k": 1}) # k=1, one doc

In [49]:
docs = retriever.get_relevant_documents("What is Task Decomposition?")

In [51]:
len(docs),docs

(1,
 [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first s

In [52]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [54]:
# Chain
chain = prompt | model

In [55]:
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

AIMessage(content='According to the provided context, Task Decomposition is the process of breaking down a complicated task into smaller and simpler steps.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 334, 'total_tokens': 359, 'completion_time': 0.020833333, 'prompt_time': 0.075507598, 'queue_time': 0.018849735000000006, 'total_time': 0.096340931}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-0ee1bd83-aa3d-4b61-9123-9e6c85a91803-0', usage_metadata={'input_tokens': 334, 'output_tokens': 25, 'total_tokens': 359})

In [67]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'According to the context, Task Decomposition is a process that involves breaking down a complicated task into smaller and simpler steps, allowing an agent to plan ahead and understand the task better.'

In [64]:
chain = prompt | model|StrOutputParser()


In [65]:
chain.invoke({"context":docs,"question":"What is Task decomposition"})


'According to the text, Task decomposition is the process of breaking down a complicated task into smaller and simpler steps. It is achieved through various techniques, including Chain of Thought (CoT), Tree of Thoughts (Yao et al. 2023), and simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", or using task-specific instructions.'